<a href="https://colab.research.google.com/github/Peppecoding/Customer-Value-Segmentation-using-RFM-Model/blob/main/Customer_Value_Segmentation_using_RFM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OBJECTIVE


---

## Segment customers based on their Recency, Frequency, and Monetary values to identify the most valuable, inactive, and at-risk clients.

---



## LOAD AND CLEAN THE DATA

In [6]:
import pandas as pd

# Si subiste un Excel
df = pd.read_excel('/content/Online Retail.xlsx')

# Si subiste un CSV (usa esta línea en su lugar)
# df = pd.read_csv('/content/Online Retail.csv')

# Ver las primeras filas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB



## What are we doing nex?

*   RFM analysis is customer-level, and rows without a CustomerID can't be assigned to any customer. We need to remove CustomerID beacuse keeping them would create wrong results.

*   We need to calculate how much each customer spent per order. This is essential to calculate the Monetary value for the RFM model. It shows how valuable a customer is.



In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:


# Drop rows with missing CustomerID
df = df.dropna(subset=['CustomerID'])

# Create TotalPrice column
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

df.head()

<ipython-input-7-292458020674>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalPrice'] = df['Quantity'] * df['UnitPrice']


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [8]:
# 1. Set snapshot date (day after the last purchase in dataset)
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# 2. Group by CustomerID and calculate RFM
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',                                    # Frequency
    'TotalPrice': 'sum'                                        # Monetary
}).reset_index()

# 3. Rename columns for clarity
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# 4. Preview the results
rfm.head()


,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,2,0.00
1,12347.0,2,7,4310.00
2,12348.0,75,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,310,1,334.40


I am assigning RFM scores to each customer to classify and better understand customer behavior. This scoring helps identify:


*   Top customers who purchase frequently, spend the most, and have bought recently.
*   At-risk customers who used to buy but haven’t returned in a while.
*   Low-value customers with low engagement and spending.













In [10]:
# Score Recency (low recency = better score)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

# Score Frequency (more orders = better score)
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])

# Score Monetary (more money = better score)
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# Create combined RFM Score
rfm['RFM_Score'] = (
    rfm['R_Score'].astype(str) +
    rfm['F_Score'].astype(str) +
    rfm['M_Score'].astype(str)
)

# Preview
rfm.head()


,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
0,12346.0,326,2,0.00,1,2,1,121
1,12347.0,2,7,4310.00,5,4,5,545
2,12348.0,75,4,1797.24,2,3,4,234
3,12349.0,19,1,1757.55,4,1,4,414
4,12350.0,310,1,334.40,1,1,2,112


In [11]:
def segment_customer(row):
    rfm_score = row['RFM_Score']
    if rfm_score == '555':
        return 'Champion'
    elif row['R_Score'] == '5':
        return 'Loyal'
    elif row['R_Score'] == '1' and row['F_Score'] in ['1', '2']:
        return 'At Risk'
    elif row['R_Score'] == '1' and row['M_Score'] == '1':
        return 'Lost'
    elif row['F_Score'] == '5' and row['M_Score'] == '5':
        return 'Big Spender'
    else:
        return 'Others'

# Apply the function
rfm['Segment'] = rfm.apply(segment_customer, axis=1)

# Preview
rfm[['CustomerID', 'RFM_Score', 'Segment']].head(10)


,CustomerID,RFM_Score,Segment
0,12346.0,121,Others
1,12347.0,545,Others
2,12348.0,234,Others
3,12349.0,414,Others
4,12350.0,112,Others
5,12352.0,354,Others
6,12353.0,111,Others
7,12354.0,114,Others
8,12355.0,112,Others
9,12356.0,435,Others


In [12]:
rfm.to_csv('rfm_segmented.csv', index=False)

In [13]:
rfm

,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment
0,12346.0,326,2,0.00,1,2,1,121,Others
1,12347.0,2,7,4310.00,5,4,5,545,Others
2,12348.0,75,4,1797.24,2,3,4,234,Others
3,12349.0,19,1,1757.55,4,1,4,414,Others
4,12350.0,310,1,334.40,1,1,2,112,Others
...,...,...,...,...,...,...,...,...,...
4367,18280.0,278,1,180.60,1,2,1,121,Others
4368,18281.0,181,1,80.82,1,2,1,121,Others
4369,18282.0,8,3,176.60,5,3,1,531,Others
4370,18283.0,4,16,2094.88,5,5,5,555,Champion


In [18]:
def segment_customer(row):
    r = str(row['R_Score'])
    f = str(row['F_Score'])
    m = str(row['M_Score'])
    rfm_score = row['RFM_Score']

    if rfm_score == '555':
        return 'Champion'
    elif r == '5' and f in ['4', '5']:
        return 'Loyal'
    elif r in ['3', '4', '5'] and f in ['1', '2']:
        return 'Potential Loyalist'
    elif r in ['2', '3'] and m in ['3', '4', '5']:
        return 'Need Attention'
    elif r == '1' and f in ['4', '5']:
        return 'At Risk'
    elif r == '1' and m == '1':
        return 'Lost'
    else:
        return 'Others'



In [19]:
rfm['Segment'] = rfm.apply(segment_customer, axis=1)

In [20]:
rfm['Segment']

,Segment
0,Lost
1,Loyal
2,Need Attention
3,Potential Loyalist
4,Others
...,...
4367,Lost
4368,Lost
4369,Others
4370,Champion


In [21]:
rfm.to_csv('rfm_segmented.csv', index=False)

# [DATA VISUALISATION ](https://lookerstudio.google.com/u/0/reporting/b2a69e8f-ae07-422d-91ee-208f0cb87078/page/W6XLF/edit)